In [1]:
from alibi.explainers import AnchorTabular
import time
from sklearn.model_selection import  train_test_split
import warnings
warnings.filterwarnings("ignore")
import sys
import os
from sklearn.model_selection import  cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score
sys.path.append(os.path.abspath('/Users/supriyasindigerekumaraswmamy/Desktop/Thesis/wind_Turbine'))
from utils.helper import *

2024-08-16 09:36:19.414980: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/supriyasindigerekumaraswmamy/Desktop/Thesis/wind_Turbine/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_data():
    failures = load_failures_data('../data/model_data/failures.csv')
    components = failures['component'].unique()
    component_data = load_all_component_data(components)
    data_splits = prepare_all_data_for_training(component_data, "target_class")
    models = load_all_models(components, "xgb")
    selected_features_data = fit_and_select_features(models, data_splits)
    models = retrain_models_on_selected_features(models, selected_features_data, data_splits)

    return models, components,selected_features_data,data_splits

In [ ]:
models,components, selected_features,data_splits = get_data()

In [13]:
def select_features(models, data_splits):
    selected_features_data = {}
    for component, model in models.items():
        X_train, X_test, y_train, y_test = data_splits[component]
        model.fit(X_train, y_train)
        selector =  model.named_steps['feature_selection']
        selected_features = X_train.columns[selector.get_support()]
        selected_features_train = X_train[selected_features]
        selected_features_train.reset_index(drop=True, inplace=True)
        selected_features_test = X_test[selected_features]
        selected_features_test.reset_index(drop=True, inplace=True)
        selected_features_data[component] = (selected_features_train, selected_features_test, selected_features)
    return selected_features_data

In [14]:
selected_features = select_features(models, data_splits)

In [6]:
for component in components:
    globals()[f"{component}_X_train"] = selected_features[component][0]
    globals()[f"{component}_X_test"] = selected_features[component][1]
    globals()[f"{component}_y_train"] = data_splits[component][2]
    globals()[f"{component}_y_test"] = data_splits[component][3]

In [7]:
for component in components:
    models[component].fit(selected_features[component][0] ,globals()[f"{component}_y_train"])

In [8]:
# confusion matrix
from sklearn.metrics import confusion_matrix

for component in components:
    globals()[f"{component}_y_pred"] = models[component].predict(selected_features[component][1])
    globals()[f"{component}_conf_matrix"] = confusion_matrix(globals()[f"{component}_y_test"], globals()[f"{component}_y_pred"])
    print(globals()[f"{component}_conf_matrix"])

[[679  10]
 [ 36   4]]
[[639  12]
 [ 44  34]]
[[617  17]
 [ 60  35]]
[[688   8]
 [ 18  15]]
[[659  18]
 [ 23  29]]


In [9]:
def train_run(X, y, X_valid, y_valid, model):
    clf = model
    clf.fit(X, y)
    y_valid_pred = clf.predict(X_valid)
    y_train_pred = clf.predict(X)
    f1_train = f1_score(y, y_train_pred, average="weighted")
    precision_train = precision_score(y, y_train_pred, average="weighted")
    recall_train = recall_score(y, y_train_pred, average="weighted")
    f1 = f1_score(y_valid, y_valid_pred, average="weighted")
    precision = precision_score(y_valid, y_valid_pred, average="weighted")
    recall = recall_score(y_valid, y_valid_pred, average="weighted")

    scores = {
        "f1_train": f1_train,
        "precision_train": precision_train,
        "recall_train": recall_train,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

    return scores

In [10]:
for component in components:
    globals()[f"{component}_scores"] = train_run(selected_features[component][0], globals()[f"{component}_y_train"], selected_features[component][1], globals()[f"{component}_y_test"], models[component])
    print(f"{component}_scores: {globals()[f'{component}_scores']}")

GEARBOX_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.9222933495910178, 'precision': 0.9132203872944614, 'recall': 0.9368998628257887}
GENERATOR_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.9141918487192088, 'precision': 0.9145592144368501, 'recall': 0.9231824417009602}
HYDRAULIC_GROUP_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.8806596320227418, 'precision': 0.8803199427549775, 'recall': 0.8943758573388203}
GENERATOR_BEARING_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.9612774980899882, 'precision': 0.9599131641118201, 'recall': 0.9643347050754458}
TRANSFORMER_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.9424417963921411, 'precision': 0.9413631386609856, 'recall': 0.943758573388203}
